## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load = "../Weather_Database/WeatherPy_Database.csv"
weather_df = pd.read_csv(file_to_load)
weather_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Caravelas,BR,-17.7125,-39.2481,83.43,60,84,9.75,broken clouds
1,1,Whitianga,NZ,-36.8333,175.7000,62.01,77,24,4.00,few clouds
2,2,Tema,GH,5.6670,-0.0166,89.60,62,0,17.27,clear sky
3,3,Phan Thiet,VN,10.9333,108.1000,77.38,87,97,10.58,overcast clouds
4,4,Arlit,NE,18.7369,7.3853,93.88,4,0,2.19,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 69
What is the maximum temperature you would like for your trip? 89


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = weather_df.loc[(weather_df["Max Temp"] <= max_temp) & (weather_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Caravelas,BR,83.43,broken clouds,-17.7125,-39.2481,
3,Phan Thiet,VN,77.38,overcast clouds,10.9333,108.1000,
5,Avarua,CK,80.60,broken clouds,-21.2078,-159.7750,
6,Saldanha,ZA,75.20,broken clouds,-33.0117,17.9442,
8,Namibe,AO,79.70,overcast clouds,-15.1961,12.1522,
11,Lamu,KE,84.20,clear sky,-2.2717,40.9020,
13,Butaritari,KI,81.23,light rain,3.0707,172.7902,
19,Doctor Juan Leon Mallorquin,PY,86.00,clear sky,-25.4000,-55.2667,
21,Rikitea,PF,77.13,scattered clouds,-23.1203,-134.9692,
26,Bredasdorp,ZA,75.20,overcast clouds,-34.5322,20.0403,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(url, params = params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (IndexError):
        print(index)
        print("Hotel not found... skipping.")
        

28
Hotel not found... skipping.
36
Hotel not found... skipping.
109
Hotel not found... skipping.
123
Hotel not found... skipping.
155
Hotel not found... skipping.
161
Hotel not found... skipping.
164
Hotel not found... skipping.
216
Hotel not found... skipping.
239
Hotel not found... skipping.
273
Hotel not found... skipping.
376
Hotel not found... skipping.
391
Hotel not found... skipping.
399
Hotel not found... skipping.
508
Hotel not found... skipping.
510
Hotel not found... skipping.
541
Hotel not found... skipping.
595
Hotel not found... skipping.


In [40]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Caravelas,BR,83.43,broken clouds,-17.7125,-39.2481,Pousada dos Navegantes
3,Phan Thiet,VN,77.38,overcast clouds,10.9333,108.1000,Hotel Saigon-PT
5,Avarua,CK,80.60,broken clouds,-21.2078,-159.7750,Paradise Inn
6,Saldanha,ZA,75.20,broken clouds,-33.0117,17.9442,Blue Bay Lodge
8,Namibe,AO,79.70,overcast clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
11,Lamu,KE,84.20,clear sky,-2.2717,40.9020,Lamu House
13,Butaritari,KI,81.23,light rain,3.0707,172.7902,Isles Sunset Lodge
19,Doctor Juan Leon Mallorquin,PY,86.00,clear sky,-25.4000,-55.2667,Hotel Puma S.A.
21,Rikitea,PF,77.13,scattered clouds,-23.1203,-134.9692,Pension Maro'i
26,Bredasdorp,ZA,75.20,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor


In [41]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [54]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dd><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [67]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure(zoom_level=2.05, center=(30.0, 10.0))
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))